In [1]:
!pip install gensim nltk scikit-learn pyLDAvis wordcloud matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you ha

In [1]:
# ==============================================================================
# 1. SETUP & DATA LOADING
# ==============================================================================
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import warnings

# Suppress deprecation warnings for cleaner output
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Download the NLTK stopwords list (only needs to be done once)
try:
    stopwords.words('english')
except LookupError:
    print("Downloading NLTK stopwords...")
    nltk.download('stopwords')

# Here is our new, smaller sample dataset
sample_documents = [
    "The solar system consists of the sun and the objects that orbit it.",
    "Mars is the fourth planet from the sun and is known as the Red Planet.",
    "The new AI model can generate realistic images from text descriptions.",
    "Machine learning is a subfield of artificial intelligence.",
    "The sun's energy is essential for life on Earth.",
    "Deep learning models require large amounts of data for training.",
    "Jupiter is the largest planet in our solar system.",
    "Natural language processing is a key area of AI research.",
    "The Apollo mission successfully landed humans on the moon, which orbits the Earth."
]

documents = sample_documents
print(f"Successfully loaded {len(documents)} custom documents.")


# ==============================================================================
# 2. PREPROCESSING THE TEXT
# ==============================================================================
print("\nPreprocessing text data...")
# Get the list of English stop words
stop_words = stopwords.words('english')

def preprocess_text(text):
    """
    Performs the following steps:
    1. Tokenizes the text (splits into words) and converts to lowercase.
    2. Removes stop words.
    3. Removes words that are 3 characters or less.
    """
    result = []
    for token in simple_preprocess(text):
        if token not in stop_words and len(token) > 3:
            result.append(token)
    return result

# Apply the preprocessing function to each document in the corpus
processed_docs = [preprocess_text(doc) for doc in documents]
print("Preprocessing complete.")
# Example of a preprocessed document
print("\n--- Example of a processed document ---")
print(processed_docs[0])


# ==============================================================================
# 3. APPLYING LDA
# ==============================================================================
# Create the dictionary and corpus needed for the LDA model

# Create a dictionary mapping words to unique IDs
id2word = Dictionary(processed_docs)

# *MODIFIED*: Since our dataset is small, we only filter words that appear less than 1 time.
id2word.filter_extremes(no_below=1, no_above=0.8)

# Convert the processed documents into a Bag-of-Words (BoW) corpus
corpus = [id2word.doc2bow(doc) for doc in processed_docs]

# --- Build the LDA Model ---
print("\nBuilding the LDA model...")

# *MODIFIED*: Set the number of topics to 3, matching our sample data's themes.
num_topics = 3

# The core LDA model training step
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=num_topics,
                     random_state=100,  # for reproducibility
                     passes=15)         # more passes can help on small data

print("LDA model training complete.")

# --- View the Topics ---
print("\n--- Top words for each of the {} identified topics ---".format(num_topics))
topics = lda_model.print_topics(num_words=5)
for i, topic in enumerate(topics):
    print(f"Topic {i+1}: {topic[1]}")


# ==============================================================================
# 4. VISUALIZE THE TOPICS
# ==============================================================================
print("\nPreparing visualization...")

# Enable the pyLDAvis visualization in the Jupyter Notebook
pyLDAvis.enable_notebook()

# Create the visualization data from the LDA model
vis_data = gensimvis.prepare(lda_model, corpus, id2word)

print("Displaying interactive topic model visualization below.")

# Display the visualization
vis_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Successfully loaded 9 custom documents.

Preprocessing text data...
Preprocessing complete.

--- Example of a processed document ---
['solar', 'system', 'consists', 'objects', 'orbit']

Building the LDA model...
LDA model training complete.

--- Top words for each of the 3 identified topics ---
Topic 1: 0.073*"planet" + 0.051*"solar" + 0.051*"system" + 0.029*"model" + 0.029*"realistic"
Topic 2: 0.043*"learning" + 0.043*"humans" + 0.043*"orbits" + 0.043*"successfully" + 0.043*"apollo"
Topic 3: 0.070*"earth" + 0.070*"energy" + 0.070*"life" + 0.070*"essential" + 0.018*"learning"

Preparing visualization...
Displaying interactive topic model visualization below.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.095255 -0.011179       1        1  55.809153
1      0.088143 -0.014122       2        1  32.881304
2      0.007112  0.025301       3        1  11.309543, topic_info=         Term      Freq     Total Category  logprob  loglift
20      earth  1.000000  1.000000  Default  30.0000  30.0000
21     energy  0.000000  0.000000  Default  29.0000  29.0000
23       life  0.000000  0.000000  Default  28.0000  28.0000
22  essential  0.000000  0.000000  Default  27.0000  27.0000
26       deep  1.000000  1.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
14       text  0.101251  1.106907   Topic3  -4.0425  -0.2122
17   learning  0.101628  1.646339   Topic3  -4.0387  -0.6055
4      system  0.101301  1.726421   Topic3  -4.0420  -0.6562
8      planet  0.101298  2.345998   Topic3  -4.0420  -0.9629
3       solar  0.101292  1.726430   Topic3  -4.0420  -0.6563

[135 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
24        2  0.972626       amounts
38        2  0.972497        apollo
33        1  0.903623          area
15        1  0.903487    artificial
0         1  0.903482      consists
25        2  0.972579          data
26        2  0.972605          deep
9         1  0.903421  descriptions
20        2  0.751515         earth
5         1  0.903477        fourth
10        1  0.903418      generate
39        2  0.972495        humans
11        1  0.903418        images
16        1  0.903490  intelligence
31        1  0.903469       jupiter
6         1  0.903476         known
40        2  0.972495        landed
34        1  0.903594      language
27        2  0.972557         large
32        1  0.903473       largest
17        1  0.607408      learning
17        2  0.607408      learning
18        1  0.903496       machine
7         1  0.903472          mars
41        2  0.972496       mission
12        1  0.903415         model
28        2  0.972591        models
42        2  0.972497          moon
35        1  0.903670       natural
1         1  0.903476       objects
2         1  0.903476         orbit
43        2  0.972496        orbits
8         1  0.852516        planet
36        1  0.903594    processing
13        1  0.903416     realistic
29        2  0.972609       require
37        1  0.903575      research
3         1  0.579230         solar
19        1  0.903490      subfield
44        2  0.972495  successfully
4         1  0.579233        system
14        1  0.903418          text
30        2  0.972605      training, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])